In [6]:
import os
import shutil
import pandas as pd

# Пути
csv_path = r"C:\Users\rusla\OneDrive\Рабочий стол\R2 datasets_tumor_only_v2.csv"
source_folder = r"C:\Users\rusla\OneDrive\Рабочий стол\База"
destination_folder = r"C:\Users\rusla\OneDrive\Рабочий стол\База_с_типом_рака"

# Убедимся, что папка назначения существует
os.makedirs(destination_folder, exist_ok=True)

# Чтение CSV с разделителем ';'
df = pd.read_csv(csv_path, sep=';')
df.columns = df.columns.str.strip()

# Удаляем начальный путь и оставляем только имя файла
df['Filename'] = df['Dataset_file'].str.replace(r'^C:\\Lab\\Python\\Datasets\\R2 MAS5-0 u133p2 HUGO log2\\', '', regex=True)
df['Filename'] = df['Filename'].str.strip()

# Создаем словарь: имя файла → тип опухоли
filename_to_type = dict(zip(df['Filename'], df['Tumor_type']))

# Получаем список файлов из папки
source_files = os.listdir(source_folder)

# Списки для отчёта
not_matched_files = []
not_matched_datasets = df['Filename'].tolist()

# Обработка файлов
for file in source_files:
    if file in filename_to_type:
        tumor_type = filename_to_type[file]
        new_name = f"{tumor_type} - {file}"
        src_path = os.path.join(source_folder, file)
        dst_path = os.path.join(destination_folder, new_name)
        shutil.copy2(src_path, dst_path)
        not_matched_datasets.remove(file)
    else:
        not_matched_files.append(file)

# Вывод результата
print("\n✅ Файлы успешно скопированы.")
print("\n❗ Файлы в папке, для которых не нашлось соответствия в таблице:")
for f in not_matched_files:
    print("-", f)

print("\n❗ Записи в таблице, для которых не найдено файла в папке:")
for f in not_matched_datasets:
    print("-", f)



✅ Файлы успешно скопированы.

❗ Файлы в папке, для которых не нашлось соответствия в таблице:
- Normal B cell (CLL) - Regev - 33.txt
- Normal Brain glioma - Sun - 23.txt
- Normal Brain regions - Berchtold - 172.txt
- Normal Colon - Marra - 32.txt
- Normal Colon - Skrzypczak - 34.txt
- Normal Endothelial cells (HUAEC HUVEC) - Luttun - 38.txt
- Normal Ewing Sarcoma - Savola - 18.txt
- Normal HCC - Wu - 39.txt
- Normal Leukemia - MILE - 77.txt
- Normal Leukocytes subtypes - Matthes - 33.txt
- Normal lung - Hou - 65.txt
- Normal Lymphocytes (TPTL) - Lye - 154.txt
- Normal Macrophage - Salazar - 44.txt
- Normal Monocytes PBM - NA - 26.txt
- Normal Muscle - Gordon - 22.txt
- Normal Muscle Skeletal - Asmann - 40.txt
- Normal Ovarian - Birrer - 10.txt
- Normal pancreatic - Wang - 51.txt
- Normal pancreatic ductal - Badea - 39.txt
- Normal Renal - Kort - 12.txt
- Normal Tumor Lung - Farez_Vidal - 45.txt
- Normal Ulcerative Colitis Colon Cancer - Watanabe - 53.txt
- Tumor Bladder - Riester - 93

In [8]:
import os

# Путь к папке с файлами
folder_path = r"C:\Users\rusla\OneDrive\Рабочий стол\База_с_типом_рака"

def process_file_inplace(file_path):
    # Читаем файл и фильтруем строки
    with open(file_path, 'r') as f:
        lines = [line for line in f if not line.startswith('#')]
    
    if not lines:
        print(f"Файл {os.path.basename(file_path)} пуст после удаления метаданных.")
        return
    
    # Перезаписываем исходный файл
    with open(file_path, 'w') as f:
        f.writelines(lines)
    
    # Выводим первые 3 строки для проверки
    print(f"\nФайл: {os.path.basename(file_path)}")
    print("Первые 3 строки после обработки:")
    for line in lines[:3]:
        print(line.strip())

# Обрабатываем все файлы в папке
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):  # Игнорируем подпапки
        process_file_inplace(file_path)

print("\nВсе файлы перезаписаны (метаданные удалены).")


Файл: ALL - Tumor ALL (B) - Carroll - 98.txt
Первые 3 строки после обработки:
A1BG	229819_at	4	4.77610398807316	2.03562390973072	5.00450139223494	3.35049724708413	4.84799690655495	4.00898878322726	4.47897180503294	2.76553474636298	3.75488750216347	2.8073549220576	4.51096191927738	1.0703893278914	5.90207357931074	1.32192809488736	5.20554891117303	6.43629511983936	5.97498811199193	6.68369645430652	-0.15200309344505	3.96347412397489	5.55458885167764	3.59693514238723	5.03562390973072	2.96347412397489	4.60288440871842	4.91169158187234	4.66106547980695	5.18586654531133	5.4093909361377	1.0703893278914	5.57288966842058	5.61765111942733	3.12101540096137	2.78659636189081	2.88752527074159	5.78920757534735	2.53605290024021	1.0703893278914	5.94485844580754	3.44625622988956	0.678071905112638	6.31650781910311	4.16992500144231	2.16992500144231	5.35049724708413	5.34340782229781	3.96347412397489	6.02680005934372	6.85673675611911	3.0703893278914	4.63226821549951	6.32732808262995	4.96347412397489	6.06177

In [9]:
import os
import pandas as pd

# Путь к папке с файлами
folder_path = r"C:\Users\rusla\OneDrive\Рабочий стол\База_с_типом_рака"

def process_file(file_path):
    # Читаем файл, пропуская строки с метаданными
    with open(file_path, 'r') as f:
        lines = [line.strip().split('\t') for line in f if not line.startswith('#')]
    
    if not lines:
        print(f"Файл {os.path.basename(file_path)} пуст после удаления метаданных.")
        return
    
    # Удаляем второй столбец (индекс 1)
    processed_lines = []
    for line in lines:
        if len(line) > 1:  # Проверяем, что столбец существует
            new_line = line[:1] + line[2:]  # Оставляем только первый и все после второго столбца
            processed_lines.append('\t'.join(new_line) + '\n')
    
    # Перезаписываем исходный файл
    with open(file_path, 'w') as f:
        f.writelines(processed_lines)
    
    # Выводим первые 3 строки для проверки
    print(f"\nФайл: {os.path.basename(file_path)}")
    print("Первые 3 строки после удаления столбца с probeset:")
    for line in processed_lines[:3]:
        print(line.strip())

# Обрабатываем все файлы в папке
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):  # Работаем только с файлами
        process_file(file_path)

print("\nВсе файлы перезаписаны (удален столбец с probeset).")


Файл: ALL - Tumor ALL (B) - Carroll - 98.txt
Первые 3 строки после удаления столбца с probeset:
A1BG	4	4.77610398807316	2.03562390973072	5.00450139223494	3.35049724708413	4.84799690655495	4.00898878322726	4.47897180503294	2.76553474636298	3.75488750216347	2.8073549220576	4.51096191927738	1.0703893278914	5.90207357931074	1.32192809488736	5.20554891117303	6.43629511983936	5.97498811199193	6.68369645430652	-0.15200309344505	3.96347412397489	5.55458885167764	3.59693514238723	5.03562390973072	2.96347412397489	4.60288440871842	4.91169158187234	4.66106547980695	5.18586654531133	5.4093909361377	1.0703893278914	5.57288966842058	5.61765111942733	3.12101540096137	2.78659636189081	2.88752527074159	5.78920757534735	2.53605290024021	1.0703893278914	5.94485844580754	3.44625622988956	0.678071905112638	6.31650781910311	4.16992500144231	2.16992500144231	5.35049724708413	5.34340782229781	3.96347412397489	6.02680005934372	6.85673675611911	3.0703893278914	4.63226821549951	6.32732808262995	4.96347412397489

In [10]:
import os
import pandas as pd
from collections import defaultdict

# Пути
input_folder = r"C:\Users\rusla\OneDrive\Рабочий стол\База_с_типом_рака"
output_folder = os.path.join(input_folder, "Объединенные_по_типу_рака")
os.makedirs(output_folder, exist_ok=True)

# Словарь для хранения данных по типам рака
cancer_data = defaultdict(list)
file_info = defaultdict(list)

# Функция для извлечения типа рака из имени файла
def get_cancer_type(filename):
    # Берем часть до первого '-'
    return filename.split(' - ')[0]

# Обработка всех файлов
for filename in os.listdir(input_folder):
    if not filename.endswith('.txt'):
        continue
    
    filepath = os.path.join(input_folder, filename)
    cancer_type = get_cancer_type(filename)
    
    try:
        # Читаем файл, пропуская метаданные
        df = pd.read_csv(filepath, sep='\t', header=None, comment='#')
        
        # Сохраняем данные
        cancer_data[cancer_type].append(df)
        file_info[cancer_type].append(filename)
        
    except Exception as e:
        print(f"Ошибка при обработке файла {filename}: {e}")

# Обработка и сохранение по типам рака
stats = {}

for cancer_type, dfs in cancer_data.items():
    # Объединяем данные
    merged_df = pd.concat([df.set_index(0) for df in dfs], axis=1)
    merged_df = merged_df.reset_index().rename(columns={'index': 'Gene'})
    
    # Сохраняем
    output_path = os.path.join(output_folder, f"{cancer_type}.txt")
    merged_df.to_csv(output_path, sep='\t', index=False)
    
    # Собираем статистику
    genes_in_all = set.intersection(*[set(df[0]) for df in dfs])
    genes_in_some = set.union(*[set(df[0]) for df in dfs]) - genes_in_all
    
    stats[cancer_type] = {
        'files': file_info[cancer_type],
        'total_genes': len(merged_df),
        'genes_in_all_files': len(genes_in_all),
        'genes_in_some_files': len(genes_in_some),
        'missing_genes_per_file': {
            fname: set(df[0]) - genes_in_all
            for fname, df in zip(file_info[cancer_type], dfs)
        }
    }
    
    # Выводим первые 3 строки
    print(f"\nТип рака: {cancer_type}")
    print(f"Файл: {output_path}")
    print("Первые 3 строки:")
    print(merged_df.head(3).to_string(index=False))

# Выводим статистику
print("\n=== Статистика ===")
for cancer_type, stat in stats.items():
    print(f"\nТип рака: {cancer_type}")
    print(f"Файлов: {len(stat['files'])}")
    print(f"Всего генов: {stat['total_genes']}")
    print(f"Генов во всех файлах: {stat['genes_in_all_files']}")
    print(f"Генов только в некоторых файлах: {stat['genes_in_some_files']}")
    
    print("\nФайлы, вошедшие в объединение:")
    for f in stat['files']:
        print(f" - {f}")
    
    print("\nГены, отсутствующие в некоторых файлах:")
    for fname, genes in stat['missing_genes_per_file'].items():
        if genes:
            print(f"В файле {fname} отсутствуют {len(genes)} генов, например: {list(genes)[:3]}...")

print(f"\nОбъединенные файлы сохранены в: {output_folder}")


Тип рака: ALL
Файл: C:\Users\rusla\OneDrive\Рабочий стол\База_с_типом_рака\Объединенные_по_типу_рака\ALL.txt
Первые 3 строки:
     0        1        2        3        4        5        6        7        8        9        10       11       12       13       14       15       16       17       18       19        20       21       22       23       24       25       26       27       28       29       30       31       32       33       34       35       36       37       38       39       40       41       42       43       44       45       46       47       48       49       50       51       52       53       54       55       56       57       58       59       60       61       62       63       64       65       66       67       68       69       70       71       72       73       74       75       76       77       78       79       80       81       82       83       84       85       86       87       88       89       90       91       92       93       94       95       96 

In [12]:
import os
import pandas as pd
from collections import defaultdict

# Пути
input_folder = r"C:\Users\rusla\OneDrive\Рабочий стол\База_с_типом_рака"
output_folder = os.path.join(input_folder, "Объединенные_по_типу_рака")
os.makedirs(output_folder, exist_ok=True)

# Словарь для хранения данных
cancer_data = defaultdict(list)
file_info = defaultdict(list)

def get_cancer_type(filename):
    """Извлекает тип рака из имени файла (часть до первого '-')"""
    return filename.split(' - ')[0]

# Обработка файлов
for filename in os.listdir(input_folder):
    if not filename.endswith('.txt'):
        continue
    
    filepath = os.path.join(input_folder, filename)
    cancer_type = get_cancer_type(filename)
    
    try:
        # Чтение файла без метаданных
        df = pd.read_csv(filepath, sep='\t', header=None, comment='#')
        cancer_data[cancer_type].append(df)
        file_info[cancer_type].append(filename)
    except Exception as e:
        print(f"Ошибка в файле {filename}: {e}")

# Сохранение объединенных данных
stats = {}
for cancer_type, dfs in cancer_data.items():
    # Объединение с сохранением всех генов
    merged_df = pd.concat([df.set_index(0) for df in dfs], axis=1)
    merged_df.columns = range(len(merged_df.columns))  # Убираем имена столбцов
    
    # Сохранение без индекса и нумерации
    output_path = os.path.join(output_folder, f"{cancer_type}.txt")
    merged_df.to_csv(output_path, sep='\t', index=True, header=False)
    
    # Статистика
    all_genes = set(merged_df.index)
    common_genes = set.intersection(*[set(df[0]) for df in dfs])
    stats[cancer_type] = {
        'files': file_info[cancer_type],
        'total_genes': len(all_genes),
        'common_genes': len(common_genes),
        'unique_genes': len(all_genes - common_genes)
    }
    
    # Вывод первых строк
    print(f"\nТип рака: {cancer_type}")
    print("Первые 3 гена:")
    print(merged_df.head(3).to_string(header=False))

# Статистика
print("\n=== Статистика объединения ===")
for cancer_type, stat in stats.items():
    print(f"\n* {cancer_type}:")
    print(f"  Файлов: {len(stat['files'])}")
    print(f"  Всего генов: {stat['total_genes']}")
    print(f"  Общих генов во всех файлах: {stat['common_genes']}")
    print(f"  Уникальных генов: {stat['unique_genes']}")
    print("  Использованные файлы:")
    for f in stat['files']:
        print(f"    - {f}")

print(f"\nРезультаты сохранены в: {output_folder}")


Тип рака: ALL
Первые 3 гена:
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [13]:
import os
import pandas as pd

# Путь к папке с объединенными файлами
input_folder = r"C:\Users\rusla\OneDrive\Рабочий стол\Объединенные_по_типу_рака"
# Создаем папку для результатов
output_folder = os.path.join(os.path.dirname(input_folder), "Средние_экспрессии")
os.makedirs(output_folder, exist_ok=True)

# Обрабатываем каждый файл в папке
for filename in os.listdir(input_folder):
    if filename.endswith('.txt'):
        file_path = os.path.join(input_folder, filename)
        
        # Читаем файл (первый столбец - гены, остальные - экспрессии)
        df = pd.read_csv(file_path, sep='\t', header=None, index_col=0)
        
        # Вычисляем среднее по строкам (игнорируя NaN)
        mean_expression = df.mean(axis=1, skipna=True)
        
        # Создаем DataFrame с генами и средними значениями
        result_df = pd.DataFrame({
            'Gene': mean_expression.index,
            'Mean_Expression': mean_expression.values
        })
        
        # Сохраняем результат
        output_path = os.path.join(output_folder, f"Mean_{filename}")
        result_df.to_csv(output_path, sep='\t', index=False, header=True)
        
        # Выводим информацию о файле
        print(f"Обработан файл: {filename}")
        print("Первые 3 строки результата:")
        print(result_df.head(3).to_string(index=False))
        print("\n" + "="*50 + "\n")

print(f"Все файлы сохранены в: {output_folder}")

Обработан файл: ALL.txt
Первые 3 строки результата:
    Gene  Mean_Expression
    A1BG         6.510262
A1BG-AS1         3.825910
    A1CF         5.071351


Обработан файл: AML.txt
Первые 3 строки результата:
    Gene  Mean_Expression
    A1BG         5.135369
A1BG-AS1         3.242127
    A1CF         4.767448


Обработан файл: Breast.txt
Первые 3 строки результата:
    Gene  Mean_Expression
    A1BG         5.242741
A1BG-AS1         3.889125
    A1CF         4.033047


Обработан файл: Cervix & Endo.txt
Первые 3 строки результата:
    Gene  Mean_Expression
    A1BG         4.101722
A1BG-AS1         2.504860
    A1CF         5.051536


Обработан файл: CLL.txt
Первые 3 строки результата:
    Gene  Mean_Expression
    A1BG         5.758381
A1BG-AS1         4.160992
    A1CF         4.375518


Обработан файл: CNS PNET.txt
Первые 3 строки результата:
    Gene  Mean_Expression
    A1BG         5.115911
A1BG-AS1         2.846755
    A1CF         4.363019


Обработан файл: Colorectal.txt
Пер

In [16]:
import os
import pandas as pd

# Путь к папке со средними значениями
input_folder = r"C:\Users\rusla\OneDrive\Рабочий стол\r2\Средние_экспрессии"
output_path = os.path.join(input_folder, "Combined_Mean_Expressions.csv")  # Изменили расширение на .csv

# Создаем пустой DataFrame для объединения
combined_df = pd.DataFrame()

# Обрабатываем каждый файл в папке
for filename in os.listdir(input_folder):
    if filename.startswith('Mean_') and filename.endswith('.txt'):
        # Извлекаем название типа рака из имени файла
        cancer_type = filename[5:-4]  # Удаляем 'Mean_' и '.txt'
        
        # Читаем файл
        file_path = os.path.join(input_folder, filename)
        df = pd.read_csv(file_path, sep='\t')
        
        # Добавляем данные в общий DataFrame
        if combined_df.empty:
            combined_df = df.set_index('Gene').rename(columns={'Mean_Expression': cancer_type})
        else:
            temp_df = df.set_index('Gene')
            combined_df = combined_df.join(temp_df.rename(columns={'Mean_Expression': cancer_type}), how='outer')

# Сохраняем объединенную таблицу в CSV без NA
combined_df.to_csv(output_path, sep=',', float_format='%.4f', na_rep='')

# Выводим информацию о результате
print("Объединенная таблица создана:")
print(f"Файл: {output_path}")
print("\nПервые 5 строк таблицы:")
print(combined_df.head().to_string())

print("\nСтатистика:")
print(f"Всего генов: {len(combined_df)}")
print(f"Типов рака: {len(combined_df.columns)}")
print("Названия типов рака:", list(combined_df.columns))

Объединенная таблица создана:
Файл: C:\Users\rusla\OneDrive\Рабочий стол\r2\Средние_экспрессии\Combined_Mean_Expressions.csv

Первые 5 строк таблицы:
               ALL       AML     Breast  Cervix & Endo       CLL  CNS PNET  Colorectal  Ependymoma  Ewing Sarcoma  Glioblastoma     Kidney      Liver       Lung  Lymphoma  Medulloblastoma   Myeloma  Neuroblastoma   Ovarian  Pancreatic  Rhabdoid
Gene                                                                                                                                                                                                                                                
A1BG      6.510262  5.135369   5.242741       4.101722  5.758381  5.115911    3.668242    5.240114       5.806834      4.661668   4.219037  10.973373   4.947228  5.229355         4.616292  5.313497       5.184282  4.277809    4.471801  5.219162
A1BG-AS1  3.825910  3.242127   3.889125       2.504860  4.160992  2.846755    2.133389    3.171064       4.420436  

In [18]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Пути к файлам
input_file = r"C:\Users\rusla\OneDrive\Рабочий стол\r2\Combined_Mean_Expressions.csv"
output_file = r"C:\Users\rusla\OneDrive\Рабочий стол\r2\Combined_Mean_Expressions_zscore.csv"

# Чтение данных
df = pd.read_csv(input_file, index_col='Gene')

# Расчет z-score по строкам (для каждого гена)
zscore_df = df.apply(lambda row: zscore(row, nan_policy='omit'), axis=1)

# Восстановление названий столбцов
zscore_df.columns = df.columns

# Сохранение результатов
zscore_df.to_csv(output_file, float_format='%.4f')

print(f"Z-score рассчитаны и сохранены в: {output_file}")
print("\nПервые 5 строк результата:")
print(zscore_df.head().to_string())

# Дополнительная статистика
print("\nСтатистика по z-score:")
print(f"Среднее z-score по генам:\n{zscore_df.mean(axis=1).describe()}")
print(f"\nСтандартное отклонение z-score по генам:\n{zscore_df.std(axis=1).describe()}")

Z-score рассчитаны и сохранены в: C:\Users\rusla\OneDrive\Рабочий стол\r2\Combined_Mean_Expressions_zscore.csv

Первые 5 строк результата:
               ALL       AML    Breast  Cervix & Endo       CLL  CNS PNET  Colorectal  Ependymoma  Ewing Sarcoma  Glioblastoma    Kidney     Liver      Lung  Lymphoma  Medulloblastoma   Myeloma  Neuroblastoma   Ovarian  Pancreatic  Rhabdoid
Gene                                                                                                                                                                                                                                            
A1BG      0.843720 -0.102745 -0.028881      -0.814332  0.326121 -0.116169   -1.112749   -0.030671       0.359439     -0.428835 -0.733585  3.916066 -0.232300 -0.038037        -0.460088  0.019857      -0.069083 -0.693107   -0.559560 -0.045058
A1BG-AS1  1.056652  0.228167  1.146341      -0.818012  1.532201 -0.332813   -1.345217    0.127409       1.900322     -0.532200 -0.670565 -

In [1]:
import pandas as pd

# Пути к файлам
mean_file = r"C:\Users\rusla\OneDrive\Рабочий стол\r2\Combined_Mean_Expressions.csv"
zscore_file = r"C:\Users\rusla\OneDrive\Рабочий стол\r2\Combined_Mean_Expressions_zscore.csv"
output_folder = r"C:\Users\rusla\OneDrive\Рабочий стол\r2"

# Чтение данных
mean_df = pd.read_csv(mean_file, index_col='Gene')
zscore_df = pd.read_csv(zscore_file, index_col='Gene')

# Функция для создания таблиц по типу рака
def create_cancer_table(cancer_type):
    # Выбираем данные для конкретного типа рака
    mean_col = mean_df[cancer_type]
    zscore_col = zscore_df[cancer_type]
    
    # Создаем объединенную таблицу
    result = pd.DataFrame({
        'Gene': mean_col.index,
        f'{cancer_type} Z-score': zscore_col,
        f'{cancer_type} mean expression': mean_col
    })
    
    # Удаляем строки с пропущенными значениями
    result = result.dropna()
    
    # Сохраняем файл
    output_path = f"{output_folder}\\{cancer_type}_expression_with_zscore.csv"
    result.to_csv(output_path, index=False, float_format='%.4f')
    
    return output_path

# Создаем таблицы для ALL и AML
all_path = create_cancer_table('ALL')
aml_path = create_cancer_table('AML')

print("Таблицы успешно созданы:")
print(f"ALL: {all_path}")
print(f"AML: {aml_path}")

# Выводим примеры данных
print("\nПервые 3 строки для ALL:")
print(pd.read_csv(all_path).head(3).to_string(index=False))
print("\nПервые 3 строки для AML:")
print(pd.read_csv(aml_path).head(3).to_string(index=False))

Таблицы успешно созданы:
ALL: C:\Users\rusla\OneDrive\Рабочий стол\r2\ALL_expression_with_zscore.csv
AML: C:\Users\rusla\OneDrive\Рабочий стол\r2\AML_expression_with_zscore.csv

Первые 3 строки для ALL:
    Gene  ALL Z-score  ALL mean expression
    A1BG       0.8437               6.5103
A1BG-AS1       1.0567               3.8259
    A1CF       0.0207               5.0714

Первые 3 строки для AML:
    Gene  AML Z-score  AML mean expression
    A1BG      -0.1027               5.1354
A1BG-AS1       0.2282               3.2421
    A1CF      -0.1676               4.7674
